In [4]:
#test_3 automate everything

In [5]:
#csv ->
#   Stock_Open  Stock_close  avg_compound

#For weekends ->
#   Stock_Open(last_weekday)  Stock_close(last_weekday)  avg_compound(weekend_values)

In [6]:
import requests
import pandas as pd
from io import StringIO

class StockData:
    def __init__(self, stock_name):
        self.stock_name = stock_name

    def stock_data(self):
        api_key = "COOFAIV9AV9SVV19"
        url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&outputsize=full&symbol={self.stock_name}&apikey={api_key}&datatype=csv'
        r = requests.get(url)
        return r.text

sd = StockData("NVDA")
data1 = sd.stock_data()
df = pd.read_csv(StringIO(data1))

print(df.head())
print(len(df['close']))

    timestamp     open    high     low   close     volume
0  2024-08-21  127.315  129.35  126.66  128.50  257883572
1  2024-08-20  128.400  129.88  125.89  127.25  300087415
2  2024-08-19  124.280  130.00  123.42  130.00  318333577
3  2024-08-16  121.940  125.00  121.18  124.58  302589872
4  2024-08-15  118.760  123.24  117.47  122.86  318086673
6241


In [7]:
df['close']

0       128.50
1       127.25
2       130.00
3       124.58
4       122.86
         ...  
6236     28.25
6237     29.19
6238     27.44
6239     25.00
6240     23.50
Name: close, Length: 6241, dtype: float64

In [8]:
import datetime
from datetime import date, datetime, timedelta

now = date.today()
print(now)

2024-08-22


In [9]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to C:\Users\RUTHVIK
[nltk_data]     REDDY\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [10]:
# import requests
# import csv
# from io import StringIO
# import pandas as pd
# from bs4 import BeautifulSoup
# from datetime import datetime, timedelta
# import nltk
# from nltk.sentiment.vader import SentimentIntensityAnalyzer

# import datetime
# from datetime import date, datetime, timedelta

# now = date.today()


# #NewsScraper Class
# class NewsScraper:
#     analyzer = SentimentIntensityAnalyzer()
#     avg_compound=[]
#     epub_date=[]

#     #init
#     def __init__(self, api_key, total_days):
#         self.api_key = api_key
#         self.total_days = total_days
#         self.now = datetime.now()

#     #scrape_new method
#     def scrape_news(self):
#         for i in range(int(self.total_days)):
#             avg=0

#             # request
#             url = f'https://content.guardianapis.com/search?section=business&page-size=200&from-date={self.now-timedelta(days=i)}&to-date={self.now-timedelta(days=i)}&show-fields=body&api-key={self.api_key}'

#             response = requests.get(url)
#             data = response.json()

#             # article content
#             article_content = []
#             for article in data['response']['results']:
#                 content = article.get('body', '')
#                 publication_date = article.get('webPublicationDate', '')
#                 date_string = publication_date.split('T')
#                 article_content.append(content)
#             self.epub_date.append(date_string)

#             # sentiment analysis
#             sentiment_list = []
#             for article in article_content:
#                 temp = self.analyzer.polarity_scores(article)
#                 sentiment_list.append(temp)

#             #collection of compound values
#             compound_list=[]
#             for i in sentiment_list:
#                 temp = i.get('compound')
#                 compound_list.append(temp)

#             #average of daily compound
#             sum=0
#             for i in compound_list:
#                 sum=sum+i

#             avg=sum/(len(compound_list))

#             self.avg_compound.append(avg)

#             #clear all the variables
#             avg=0
#             article_content.clear()
#             sentiment_list.clear()
#             compound_list.clear()
#             date_string=""

# # usage
# api_key = '1e78027b-d07c-4e35-9a0a-8f1d2b4e5549'

# start_date='2022-10-01'
# current_date=str(now)

# d1 = datetime.strptime(start_date, "%Y-%m-%d")
# d2 = datetime.strptime(current_date, "%Y-%m-%d")

# total_days=d2-d1

# ns = NewsScraper(api_key,total_days.days)

# ns.scrape_news()


In [28]:
import requests
import csv
from io import StringIO
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

class NewsScraper:
    analyzer = SentimentIntensityAnalyzer()
    avg_compound = []
    epub_date = []

    def __init__(self, api_key, total_days):
        self.api_key = api_key
        self.total_days = total_days
        self.now = datetime.now()

    def scrape_news(self):
        c=0
        for i in range(self.total_days):
            avg = 0
            epub_date_day = []

            date = (self.now - timedelta(days=i)).strftime('%Y-%m-%d')
            url = f'https://content.guardianapis.com/search?section=business&page-size=200&from-date={date}&to-date={date}&show-fields=body&api-key={self.api_key}'

            try:
                response = requests.get(url)
                data = response.json()
            except requests.exceptions.RequestException as e:
                print(f"Error requesting API: {e}")
                continue

            if 'response' not in data or 'results' not in data.get('response', {}):
                print(f"Error: Unexpected response format. Response: {data}")
                continue

            article_content = []
            for article in data['response']['results']:
                content = article.get('fields', {}).get('body', '')
                publication_date = article.get('webPublicationDate', '')
                date_string = publication_date.split('T')[0]  # extract the date part

                # Remove HTML tags using BeautifulSoup
                soup = BeautifulSoup(content, 'html.parser')
                content = soup.get_text()

                article_content.append(content)
                epub_date_day.append(date_string)  # append the date string

            sentiment_list = []
            for article in article_content:
                temp = self.analyzer.polarity_scores(article)
                sentiment_list.append(temp)

            compound_list = []
            for i in sentiment_list:
                temp = i.get('compound')
                compound_list.append(temp)

            if compound_list:
                sum = 0
                for i in compound_list:
                    sum = sum + i

                avg = sum / (len(compound_list))

                self.avg_compound.append(avg)
                self.epub_date.append(epub_date_day[0])

            avg = 0
            article_content.clear()
            sentiment_list.clear()
            compound_list.clear()
            epub_date_day.clear()

            c=c+1
            print(c)

api_key = '1e78027b-d07c-4e35-9a0a-8f1d2b4e5549'

start_date = '2024-08-01'
current_date = str(date.today())

d1 = datetime.strptime(start_date, "%Y-%m-%d")
d2 = datetime.strptime(current_date, "%Y-%m-%d")

total_days = (d2 - d1).days

ns = NewsScraper(api_key, total_days)
ns.scrape_news()

print(ns.avg_compound)
print(ns.epub_date)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
[-0.3117666666666667, 0.53845, 0.6196818181818181, 0.25249166666666667, 0.09830000000000004, 0.9917499999999999, 0.34463333333333335, 0.8175727272727275, 0.7319333333333332, 0.5815214285714285, 0.5064500000000001, -0.008133333333333326, 0.6578285714285714, 0.2497875, 0.33088333333333336, 0.9644875, -0.11450714285714285, -0.2614888888888889, 0.5108124999999999, 0.9907666666666667, 0.33273749999999996]
['2024-08-22', '2024-08-21', '2024-08-20', '2024-08-19', '2024-08-18', '2024-08-17', '2024-08-16', '2024-08-15', '2024-08-14', '2024-08-13', '2024-08-12', '2024-08-11', '2024-08-10', '2024-08-09', '2024-08-08', '2024-08-07', '2024-08-06', '2024-08-05', '2024-08-04', '2024-08-03', '2024-08-02']


In [29]:
print(len(ns.avg_compound))
print(len(ns.epub_date))

21
21


In [ ]:
#LSTM model


In [14]:
# import pandas as pd
# from io import StringIO
# df = pd.read_csv(StringIO(data1))

# print(df.head())
# print(len(df['close']))

In [15]:
# # Import datetime class from datetime module
# from datetime import date, datetime, timedelta

# # returns current date and time
# now = date.today()
# print(now)


In [16]:
# start_date='2022-10-01'
# current_date=str(now)

# d1 = datetime.strptime(start_date, "%Y-%m-%d")
# d2 = datetime.strptime(current_date, "%Y-%m-%d")

# total_days=d2-d1

# print(total_days.days)


In [17]:
# import requests
# import csv
# from io import StringIO
# import pandas as pd
# from bs4 import BeautifulSoup

# api_key = '1e78027b-d07c-4e35-9a0a-8f1d2b4e5549'

# for i in range(total_days.days):
#     avg=0

#     #request
#     url = f'https://content.guardianapis.com/search?section=business&page-size=200&from-date={now-timedelta(days=i)}&to-date={now-timedelta(days=i)}&show-fields=body&api-key={api_key}'

#     response = requests.get(url)
#     data=response.json()

#     #article content
#     article_content=[]

#     for article in data['response']['results']:
#         content = article.get('body', '')
#         article_content.append(content)

#     #sentiment analysis
    


In [18]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns

# from tensorflow import keras
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import LSTM


In [19]:
# df = pd.read_csv('monthly_milk_production.csv',
# 				index_col='Date',
# 				parse_dates=True)
# df.index.freq = 'MS'


In [20]:
# df.head()


In [21]:
# # Plotting graph b/w production and date
# df.plot(figsize=(12, 6))


In [22]:
# from statsmodels.tsa.seasonal import seasonal_decompose
# results = seasonal_decompose(df['Production'])
# results.plot()


In [23]:
# train = df.iloc[:156]
# test = df.iloc[156:]


In [24]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# scaler.fit(train)
# scaled_train = scaler.transform(train)
# scaled_test = scaler.transform(test)


In [25]:
# from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

# n_input = 3
# n_features = 1
# generator = TimeseriesGenerator(scaled_train,
# 								scaled_train,
# 								length=n_input,
# 								batch_size=1)
# X, y = generator[0]
# print(f'Given the Array: \n{X.flatten()}')
# print(f'Predict this y: \n {y}')
# # We do the same thing, but now instead for 12 months
# n_input = 12
# generator = TimeseriesGenerator(scaled_train,
# 								scaled_train,
# 								length=n_input,
# 								batch_size=1)


In [26]:
# # define model
# model = Sequential()
# model.add(LSTM(100, activation='relu',
# 			input_shape=(n_input, n_features)))
# model.add(Dense(1))
# model.compile(optimizer='adam', loss='mse')
# model.summary()
# model.fit(generator, epochs=5)
